<a href="https://colab.research.google.com/github/SreeBhargaviM/Wine_Predictions/blob/master/basic_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
train_data = pd.read_csv('winemag-data-130k-v2.csv')

In [25]:
train_data = train_data.sample(frac=1)

In [26]:
train_data.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [27]:
len(list(train_data['variety']))

129971

In [28]:
names = set(train_data['variety'])
len(names)

708

In [29]:
wine_dict = {x: i for i,x in enumerate(list(names))}


In [30]:
def str_to_num(s,dict1):
  l = [dict1[word] for word in s]
  return l

In [31]:
wine_label = str_to_num(list(train_data['variety']), wine_dict)

In [32]:
wine_label[1:10]

[540, 141, 141, 195, 676, 178, 571, 99, 401]

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
wine_label = tf.keras.utils.to_categorical(wine_label)

In [42]:
len(wine_label[1000])

708

In [34]:
train1 = list(train_data['description'])


In [35]:
for i in range(len(train1)):
  train1[i] = train1[i].lower()


In [36]:
train1[120]

'red cherry, a touch of cinnamon, purple flowers and baked vanilla cookies show on the nose of this wine. black cherry and gingerbread spice consume the palate, which also offers a rocky minerality and a vanilla foam finish.'

In [37]:
len(train1[120].split())

38

In [38]:
def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [word for word in sentence.split() if word not in stop_words]
        new_sent1 = [wor for wor in new_sent if wor.isalpha()]
        train_sentences[i] = ' '.join(new_sent1)
    return train_sentences

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
import numpy as np

In [39]:
stop_words = set(stopwords.words("english"))
train1 = np.array(filter_stop_words(train1, stop_words))

In [40]:
train1[120]

'red touch purple flowers baked vanilla cookies show nose black cherry gingerbread spice consume also offers rocky minerality vanilla foam'

In [41]:
len(train1[120].split())

20

In [45]:
tokenizer1 = Tokenizer( num_words = 1000, oov_token='<OOV>')
tokenizer1.fit_on_texts(train1)
word_index1 = tokenizer1.word_index
vocab_len1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(train1)
padded1 = pad_sequences(sequences1,maxlen= 25, truncating= 'post')




In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 24, input_length=25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(24, return_sequences=True),
    tf.keras.layers.LSTM(12),
    #tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dense(708, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 24)            24000     
_________________________________________________________________
batch_normalization (BatchNo (None, 25, 24)            96        
_________________________________________________________________
lstm (LSTM)                  (None, 25, 24)            4704      
_________________________________________________________________
lstm_1 (LSTM)                (None, 12)                1776      
_________________________________________________________________
dense (Dense)                (None, 708)               9204      
Total params: 39,780
Trainable params: 39,732
Non-trainable params: 48
_________________________________________________________________


In [47]:
model.fit(padded1, wine_label, epochs=10, batch_size=200 ,validation_split=0.2)

Epoch 1/10
520/520 [==============================] - 22s 42ms/step - loss: 3.9269 - accuracy: 0.1033 - val_loss: 3.8935 - val_accuracy: 0.1151
Epoch 2/10
520/520 [==============================] - 21s 40ms/step - loss: 3.7562 - accuracy: 0.1655 - val_loss: 3.5796 - val_accuracy: 0.2217
Epoch 3/10
520/520 [==============================] - 21s 40ms/step - loss: 3.3882 - accuracy: 0.2302 - val_loss: 3.2489 - val_accuracy: 0.2579
Epoch 4/10
520/520 [==============================] - 21s 40ms/step - loss: 3.1380 - accuracy: 0.2742 - val_loss: 3.0718 - val_accuracy: 0.2876
Epoch 5/10
520/520 [==============================] - 21s 41ms/step - loss: 2.9617 - accuracy: 0.2972 - val_loss: 2.9142 - val_accuracy: 0.3073
Epoch 6/10
520/520 [==============================] - 21s 40ms/step - loss: 2.8110 - accuracy: 0.3322 - val_loss: 2.7888 - val_accuracy: 0.3433
Epoch 7/10
520/520 [==============================] - 22s 41ms/step - loss: 2.6911 - accuracy: 0.3590 - val_loss: 2.6947 - val_accuracy: